In [15]:
import cv2
import os

def add_user(user_name):
    """Yeni kullanıcıyı kaydeder."""
    os.makedirs("known_faces", exist_ok=True)
    save_path = f"known_faces/{user_name}.jpg"

    video_capture = cv2.VideoCapture(0)
    if not video_capture.isOpened():
        print("Kamera açılamadı, bağlantıyı kontrol edin.")
        return

    print(f"{user_name}, lütfen kameraya bakın...")

    while True:
        ret, frame = video_capture.read()
        if not ret:
            print("Görüntü alınamadı.")
            break

        cv2.imshow("Yüz Kaydı (Kaydetmek için 's', iptal için 'q')", frame)

        key = cv2.waitKey(1)
        if key & 0xFF == ord('s'):
            cv2.imwrite(save_path, frame)
            print(f"{user_name} başarıyla kaydedildi! Dosya: {save_path}")
            break
        elif key & 0xFF == ord('q'):
            print("Kayıt iptal edildi.")
            break

    video_capture.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    while True:
        name = input("Kayıt edilecek kullanıcının adını girin (Çıkmak için 'q'): ")
        if name.lower() == 'q':
            print("Kayıt işlemi sonlandırıldı.")
            break
        elif name.strip() == "":
            print("İsim boş olamaz.")
            continue
        else:
            add_user(name)


Kayıt edilecek kullanıcının adını girin (Çıkmak için 'q'):  MERVE


MERVE, lütfen kameraya bakın...
MERVE başarıyla kaydedildi! Dosya: known_faces/MERVE.jpg


Kayıt edilecek kullanıcının adını girin (Çıkmak için 'q'):  ELON MUSK


ELON MUSK, lütfen kameraya bakın...
ELON MUSK başarıyla kaydedildi! Dosya: known_faces/ELON MUSK.jpg


Kayıt edilecek kullanıcının adını girin (Çıkmak için 'q'):  q


Kayıt işlemi sonlandırıldı.


In [23]:
import cv2
import face_recognition
import numpy as np
import os

def recognize_face():
    KNOWN_FACES_DIR = "known_faces"
    known_faces = []
    known_names = []

    for filename in os.listdir(KNOWN_FACES_DIR):
        image_path = os.path.join(KNOWN_FACES_DIR, filename)

        # Gizli dosyaları ve sistem dosyalarını atla
        if filename.startswith('.') or not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        try:
            image = face_recognition.load_image_file(image_path)
            encodings = face_recognition.face_encodings(image)

            if encodings:
                known_faces.append(encodings[0])
                known_names.append(os.path.splitext(filename)[0])
            else:
                print(f"Uyarı: {filename} içinde yüz bulunamadı.")
        except Exception as e:
            print(f"Resim yüklenirken bir hata oluştu: {filename}. Hata: {e}")

    # Kamera ile yüz tanıma işlemi
    video_capture = cv2.VideoCapture(0)
    if not video_capture.isOpened():
        print("Kamera açılamadı.")
        return "Kamera hatası"

    name = "Erişim reddedildi"
    face_detected = False

    while not face_detected:
        ret, frame = video_capture.read()
        if not ret:
            print("Görüntü alınamadı.")
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        for face_encoding, (top, right, bottom, left) in zip(face_encodings, face_locations):
            matches = face_recognition.compare_faces(known_faces, face_encoding)
            face_distances = face_recognition.face_distance(known_faces, face_encoding)
            best_match_index = np.argmin(face_distances) if matches else None

            if best_match_index is not None and matches[best_match_index]:
                name = known_names[best_match_index]

            # Çerçeve rengi ve etiketi ayarla
            if name != "Erişim reddedildi":
                label = "Giriş başaril"
                color = (0, 255, 0)  
            else:
                label = "Erişim reddedildi"
                color = (0, 0, 255)  

            cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
            cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
            face_detected = True

        cv2.imshow("Yüz Tanıma", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            name = "İşlem iptal edildi"
            break

    video_capture.release()
    cv2.destroyAllWindows()
    return name

# Doğrudan çalıştırılırsa:
if __name__ == "__main__":
    result = recognize_face()
    print("Tanımlanan kişi:", result)


Atlanıyor (geçersiz dosya): desktop.ini
Tanımlanan kişi: MERVE


In [25]:
import csv
from datetime import datetime
import os

LOG_FILE = "logs/access_log.csv"

# Klasör yoksa oluştur
os.makedirs("logs", exist_ok=True)

# Dosya yoksa başlıkla oluştur
if not os.path.exists(LOG_FILE):
    with open(LOG_FILE, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Tarih-Saat", "Kullanıcı"])

def log_access(user_name):
    """Kişi bilgisini tarih-saat ile CSV'ye yazar."""
    with open(LOG_FILE, mode="a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([datetime.now().strftime("%Y-%m-%d %H:%M:%S"), user_name])
    print(f"Giriş kaydedildi: {user_name}")


In [27]:
from add_user import add_user
from face_recognizer import recognize_face
from log_writer import log_access

def main():
    while True:
        print("\n--- Yüz Tanıma Sistemi ---")
        print("1. Yeni Kullanıcı Kaydı")
        print("2. Yüz Tanıma ve Giriş Kontrolü")
        print("3. Çıkış")

        choice = input("Bir seçenek girin (1/2/3): ")

        if choice == '1':
            name = input("Kullanıcının adını girin: ")
            if name.strip() == "":
                print("İsim boş olamaz.")
                continue
            add_user(name)

        elif choice == '2':
            result = recognize_face()
            log_access(result)

        elif choice == '3':
            print("Programdan çıkılıyor.")
            break

        else:
            print("Geçersiz seçenek. Lütfen tekrar deneyin.")

if __name__ == "__main__":
    main()



--- Yüz Tanıma Sistemi ---
1. Yeni Kullanıcı Kaydı
2. Yüz Tanıma ve Giriş Kontrolü
3. Çıkış


Bir seçenek girin (1/2/3):  2


UnidentifiedImageError: cannot identify image file 'C:\\Users\\LENOVO\\yuz-tanima-sistemi\\known_faces\\desktop.ini'